In [4]:
#need to run pip install praw first
import praw
import pandas as pd

In [5]:
reddit = praw.Reddit(client_id='JKPBLiybYVEN11oyZo59-Q', client_secret='-WfhbRTUWf3qEQBkZveWVfBOL_av6A', user_agent='WSBScraping')

In [10]:
posts = []
wsb_subreddit = reddit.subreddit('wallstreetbets')
! mkdir ../posts

In [7]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
data_clean = data_clean[data_clean['ETF']=='N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 9272


In [ ]:
for symbol in symbols:
    posts = []
    for post in wsb_subreddit.search(symbol, sort='hot', limit=1000):
        posts.append([post.title, post.score, post.num_comments, post.selftext, post.created])
    if posts:
        posts_df = pd.DataFrame(posts,columns=['title', 'score', 'num_comments', 'body', 'created'])
        posts_df.to_csv('../posts/{}_posts.csv'.format(symbol))

In [26]:
posts = []
for s in range(100):
    post = wsb_subreddit.random()
    posts.append([post.title, post.score, post.num_comments, post.selftext, post.created])
posts_df = pd.DataFrame(posts,columns=['title', 'score', 'num_comments', 'body', 'created'])
print(posts_df)   

                                                title  score  num_comments  \
0                                Buy More, Make More!     13             3   
1   Looking for the list of 83 stocks by Goldman S...     28             7   
2   Cover had Chicken Tendie Oil smears all over i...     26             6   
3   R.I.P for those of us who bought puts. Cup noo...    245            65   
4          Is this what they meant by Buy. HODL. DRS.      5             6   
..                                                ...    ...           ...   
95                                              ETFs?     25             2   
96              Nothing personal, it’s jist spam now.     78            24   
97  HMHC-Agreement Could very well be going throug...      4            42   
98                        r/Green lattice Targeted Us   2115           223   
99          WSB NERDS WATCHING THEIR TILES ON r/place   1296            36   

                                                 body       cre

### Tried iteratively running a url request to get all posts via a 3rd party API but kept getting "HTTPError: HTTP Error 429: Too Many Requests" error

In [52]:
after = 1648112985#1357016400 #jan 1 2013
before = 1649112985 #today
all_ids = []
import requests
while before > after:
    with urllib.request.urlopen("https://api.pushshift.io/reddit/search/submission/?subreddit=wallstreetbets&sort=desc&sort_type=created_utc&after={0}&before={1}&size=500".format(after,before)) as url:
        data = json.loads(url.read().decode())
    all_ids = all_ids + [x['id'] for x in data['data']]
    if len([x['created_utc'] for x in data['data']])==0:
        before = after
    else:
        before = [x['created_utc'] for x in data['data']][-1]


HTTPError: HTTP Error 429: Too Many Requests